In [1]:
!apt-get update -qq
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install -y
!google-chrome --version

!pip -q install --upgrade pip
!pip -q install selenium webdriver-manager pandas

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package google-chrome-stable.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (139.0.7258.138-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Errors were encountered while processing:
 google-chrome-stable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Correcting dependenci

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Chrome setup
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

all_data = []

# Loop through 61 pages
for page in range(1, 62):
    url = f"https://www.coffeereview.com/review/page/{page}/" if page > 1 else "https://www.coffeereview.com/review/"
    print(f"Scraping page {page}...")
    driver.get(url)
    time.sleep(2)

    reviews = driver.find_elements(By.CSS_SELECTOR, "div.review-template")

    for r in reviews:
        try:
            coffee_name = r.find_element(By.CSS_SELECTOR, "h2.review-title").text.strip()
        except:
            coffee_name = ""
        try:
            rating = r.find_element(By.CSS_SELECTOR, "span.review-template-rating").text.strip()
        except:
            rating = ""

        # Open review detail page
        try:
            link = r.find_element(By.CSS_SELECTOR, "h2.review-title a").get_attribute("href")
            driver.execute_script("window.open(arguments[0]);", link)
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2)

            def get_text(label):
                try:
                    return driver.find_element(By.XPATH, f"//td[contains(text(), '{label}')]/following-sibling::td").text.strip()
                except:
                    return ""

            roaster_location = get_text("Roaster Location")
            coffee_origin = get_text("Coffee Origin")
            roast_level = get_text("Roast Level")
            est_price = get_text("Price")
            review_date = get_text("Review Date")
            aroma = get_text("Aroma")
            acidity = get_text("Acidity/Structure")
            body = get_text("Body")
            flavor = get_text("Flavor")
            aftertaste = get_text("Aftertaste")

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            roaster_location = coffee_origin = roast_level = est_price = review_date = ""
            aroma = acidity = body = flavor = aftertaste = ""

        all_data.append({
            "Coffee Name": coffee_name,
            "Roaster Location": roaster_location,
            "Coffee Origin": coffee_origin,
            "Roast Level": roast_level,
            "Est. Price": est_price,
            "Review Date": review_date,
            "Aroma": aroma,
            "Acidity/Structure": acidity,
            "Body": body,
            "Flavor": flavor,
            "Aftertaste": aftertaste,
            "Rating": rating
        })

driver.quit()

# Save CSV
df = pd.DataFrame(all_data)
df.to_csv("coffee_reviews_61pages.csv", index=False)
print("Saved", len(df), "reviews to coffee_reviews_61pages.csv")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

In [3]:
from google.colab import files

files.download("coffee_reviews_61pages.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>